In [28]:
config['starrocks']['multi']

1

In [27]:
config = {'starrocks':
              {
                "host": ["192.168.50.36","192.168.50.37","192.168.50.38"], 
                "port": 19030,
                "user": "root",
                "password": "Databuff@123",
                "database": "databuff",
                "mincached": 1,
                "maxconnections": 10,
                "index.rca": "dc_rca",
                "index.aggregate": "dc_aggregate_result",
                "index.span": "dc_span",
                "multi": 1
              }
        }

In [26]:
from dbutils.pooled_db import PooledDB
import pymysql

class MultiHostDBPool():
    def __init__(self, config):
        self.config = config
        self.current_host_index = 0

        if config["starrocks"]['multi']:
            self.__pools = {}
            for index, host in enumerate(config["starrocks"]["host"]):
                # 创建每个主机的连接池
                pool = PooledDB(
                    creator = pymysql,
                    maxconnections = config["starrocks"]["maxconnections"],
                    mincached = config["starrocks"]["mincached"],
                    host = host,
                    port = config["starrocks"]["port"],
                    user = config["starrocks"]["user"],
                    password = config["starrocks"]["password"],
                    database = config["starrocks"]["database"]
                )
                self.__pools[host] = pool
        else:
             self.__pool = PooledDB(
                    creator = pymysql,
                    maxconnections = config["starrocks"]["maxconnections"],
                    mincached = config["starrocks"]["mincached"],
                    host = config["starrocks"]["host"],
                    port = config["starrocks"]["port"],
                    user = config["starrocks"]["user"],
                    password = config["starrocks"]["password"],
                    database = config["starrocks"]["database"]
                )

    def get_connection_multi(self):
        # 轮询选择主机的连接池
        for _ in range(len(self.config["starrocks"]["host"])):
            host = self.config["starrocks"]["host"][self.current_host_index]
            self.current_host_index = (self.current_host_index + 1) % len(self.config["starrocks"]["host"])
            try:
                return self.__pools[host].connection()
            except Exception as e:
                print(f"Failed to connect to {host}: {e}")
        raise Exception("Unable to connect to any host.")

    def get_connection(self):
        return self.__pool.connection()

    def release_connection(self, conn):
        # 将连接放回连接池中以供后续重用
        conn.close()

    def execute_query(self, sql):
        if self.config["starrocks"]['multi']:
            conn = self.get_connection_multi()
        else:
            conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
        cursor.close()
        self.release_connection(conn)
        return data

db_pool = MultiHostDBPool(config)
result = db_pool.execute_query("SELECT * FROM dc_span limit 1")
print(result)


()
